# NLP with oil and renewable energies project

#### Get libraries

In [ ]:
import requests
import re
import json
import sys
import os
import time
import uuid
import warnings
from bs4 import BeautifulSoup
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB  
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
import numpy as np
from PIL import Image
from pyspark.ml.feature import CountVectorizer
from sklearn.linear_model import LogisticRegression
import argparse
from matplotlib import pyplot as plt
import wordcloud
import requests, json
import pyspark
from pyspark import SparkContext
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns
import spacy
nlp = spacy.load("en_core_web_sm")
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
import re
import itertools
import unicodedata
import networkx as nx
from scipy.spatial import distance
%matplotlib inline
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

### Creating dfs

In [ ]:
corpus_oil_price=pd.read_csv('data/corpus_oil_price_3.csv', index_col=0)
corpus_alternative_energies=pd.read_csv('data/corpus_alternative_energies_3.csv', index_col=0)
corpus_crude_oil=pd.read_csv('data/corpus_crude_oil_3.csv', index_col=0)
corpus_oil_price

In [ ]:
corpus_oil_price["date_published"] = pd.to_datetime(corpus_oil_price["date_published"], utc=True)

In [ ]:
corpus_oil_price['year'] = corpus_oil_price['date_published'].dt.year

In [ ]:
filtered_df = corpus_oil_price.loc[corpus_oil_price['year']==2017]

In [ ]:
filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("[","")
filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("]","")
filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("'","")
filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("',","")
filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace(",'","")
filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace(",","")

In [ ]:
col_one_list_2 = filtered_df['most_common_words'].tolist()
col_one_list = []
for i in col_one_list_2:
    listRes = list(i.split(" "))
    col_one_list.append(listRes)   
col_one_list

In [ ]:
stopwords = ["$","d","oil","prices","price",""," "]
for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
col_one_list

In [ ]:
word_cnt = {}
for words in col_one_list:
    for word in words:
        if word not in word_cnt:
            word_cnt[word] = 1
        else:
            word_cnt[word] += 1
    
word_cnt_df = pd.DataFrame({'word': [k for k in word_cnt.keys()], 'cnt': [v for v in word_cnt.values()]})
len(word_cnt_df)  
word_cnt_df.sort_values(by=['cnt'])




In [ ]:
vocab = {}
target_words = word_cnt_df[word_cnt_df['cnt'] > 3]['word'].to_numpy()
for word in target_words:
    if word not in vocab:
        vocab[word] = len(vocab)

re_vocab = {}
for word, i in vocab.items():
    re_vocab[i] = word
    
len(vocab)

In [ ]:
combinations = [list(itertools.combinations(words, 2)) for words in col_one_list]
combination_matrix = np.zeros((len(vocab), len(vocab)))

for tweet_comb in combinations:
    for comb in tweet_comb:
        if comb[0] in target_words and comb[1] in target_words:
            combination_matrix[vocab[comb[0]], vocab[comb[1]]] += 1
            combination_matrix[vocab[comb[1]], vocab[comb[0]]] += 1
        
for i in range(len(vocab)):
    combination_matrix[i, i] /= 2
        
combination_matrix

In [ ]:
jaccard_matrix = 1 - distance.cdist(combination_matrix, combination_matrix, 'jaccard')
jaccard_matrix

In [ ]:
nodes = []

for i in range(len(vocab)):
    for j in range(i+1, len(vocab)):
        jaccard = jaccard_matrix[i, j]
        if jaccard > 0:
            nodes.append([re_vocab[i], re_vocab[j], word_cnt[re_vocab[i]], word_cnt[re_vocab[j]], jaccard])
            

nodes

In [ ]:
G = nx.Graph()
G.nodes(data=True)

for pair in nodes:
    node_x, node_y, node_x_cnt, node_y_cnt, jaccard = pair[0], pair[1], pair[2], pair[3], pair[4]
    if not G.has_node(node_x):
        G.add_node(node_x, count=node_x_cnt)
    if not G.has_node(node_y):
        G.add_node(node_y, count=node_y_cnt)
    if not G.has_edge(node_x, node_y):
        G.add_edge(node_x, node_y, weight=jaccard)
        
plt.figure(figsize=(15,15))
pos = nx.spring_layout(G, k=0.1)

node_size = [d['count']*150 for (n,d) in G.nodes(data=True)]
nx.draw_networkx_nodes(G, pos, node_color='cyan', alpha=1.0, node_size=node_size)
nx.draw_networkx_labels(G, pos)

edge_width = [d['weight']*10 for (u,v,d) in G.edges(data=True)]
nx.draw_networkx_edges(G, pos, alpha=0.1, edge_color='grey', width=edge_width)

plt.axis('off')
plt.show()

In [ ]:
for ano in range(2010, 2022):
    corpus_oil_price["date_published"] = pd.to_datetime(corpus_oil_price["date_published"], utc=True)
    corpus_oil_price['year'] = corpus_oil_price['date_published'].dt.year
    filtered_df = corpus_oil_price.loc[corpus_oil_price['year']==ano]
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("[","")
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("]","")
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("'","")
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("',","")
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace(",'","")
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace(",","")
    col_one_list_2 = filtered_df['most_common_words'].tolist()
    col_one_list = []
    for i in col_one_list_2:
        listRes = list(i.split(" "))
        col_one_list.append(listRes)   

    stopwords = ["$","d","oil","prices","price","", " "]
    for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
    
    stopwords = ["$","d","oil","prices","price","", " "]
    for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
    
    stopwords = ["$","d","oil","prices","price","", " "]
    for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
    
    word_cnt = {}
    for words in col_one_list:
        for w in words:
            if w not in word_cnt:
                word_cnt[w] = 1
            else:
                word_cnt[w] += 1
    
    word_cnt_df = pd.DataFrame({'word': [k for k in word_cnt.keys()], 'cnt': [v for v in word_cnt.values()]})
    word_cnt_df.sort_values(by=['cnt'])
    vocab = {}
    target_words = word_cnt_df[word_cnt_df['cnt'] > 3]['word'].to_numpy()
    for word in target_words:
        if word not in vocab:
            vocab[word] = len(vocab)

    re_vocab = {}
    for word, i in vocab.items():
        re_vocab[i] = word        
    
    combinations = [list(itertools.combinations(words, 2)) for words in col_one_list]
    combination_matrix = np.zeros((len(vocab), len(vocab)))

    for tweet_comb in combinations:
        for comb in tweet_comb:
            if comb[0] in target_words and comb[1] in target_words:
                combination_matrix[vocab[comb[0]], vocab[comb[1]]] += 1
                combination_matrix[vocab[comb[1]], vocab[comb[0]]] += 1
        
    for i in range(len(vocab)):
            combination_matrix[i, i] /= 2
        
    jaccard_matrix = 1 - distance.cdist(combination_matrix, combination_matrix, 'jaccard')
    nodes = []
    for i in range(len(vocab)):
        for j in range(i+1, len(vocab)):
            jaccard = jaccard_matrix[i, j]
            if jaccard > 0:
                nodes.append([re_vocab[i], re_vocab[j], word_cnt[re_vocab[i]], word_cnt[re_vocab[j]], jaccard])
    
    G = nx.Graph()
    G.nodes(data=True)
    for pair in nodes:
        node_x, node_y, node_x_cnt, node_y_cnt, jaccard = pair[0], pair[1], pair[2], pair[3], pair[4]
        if not G.has_node(node_x):
            G.add_node(node_x, count=node_x_cnt)
        if not G.has_node(node_y):
            G.add_node(node_y, count=node_y_cnt)
        if not G.has_edge(node_x, node_y):
            G.add_edge(node_x, node_y, weight=jaccard)
        
    plt.figure(figsize=(20,15))
    pos = nx.spring_layout(G, k=0.1)

    node_size = [d['count']*150 for (n,d) in G.nodes(data=True)]
    nx.draw_networkx_nodes(G, pos, node_color='cyan', alpha=1.0, node_size=node_size)
    nx.draw_networkx_labels(G, pos)

    edge_width = [d['weight']*7 for (u,v,d) in G.edges(data=True)]
    nx.draw_networkx_edges(G, pos, alpha=0.5, edge_color='grey', width=edge_width)

    plt.axis('off')
    plt.savefig('images/oil'+str(ano)+'.png', dpi=300)


In [ ]:
for ano in range(2010, 2022):
    corpus_alternative_energies["date_published"] = pd.to_datetime(corpus_alternative_energies["date_published"], utc=True)
    corpus_alternative_energies['year'] = corpus_alternative_energies['date_published'].dt.year
    filtered_df = corpus_alternative_energies.loc[corpus_alternative_energies['year']==ano]
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("[","")
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("]","")
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("'","")
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace("',","")
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace(",'","")
    filtered_df['most_common_words'] = filtered_df['most_common_words'].str.replace(",","")
    col_one_list_2 = filtered_df['most_common_words'].tolist()
    col_one_list = []
    for i in col_one_list_2:
        listRes = list(i.split(" "))
        col_one_list.append(listRes)   

    stopwords = ["$","d","oil","prices","price","", " ", "energy", "power"]
    for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
    
    stopwords = ["$","d","oil","prices","price","", " ", "energy", "power"]
    for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
    
    stopwords = ["$","d","oil","prices","price","", " ", "energy", "power"]
    for c in col_one_list:
        #for i in range(len(c)):
        #    c[i] = c[i].lower()
        for word in c:  # iterating on a copy since removing will mess things up
            if word in stopwords:
                c.remove(word)
    
    word_cnt = {}
    for words in col_one_list:
        for w in words:
            if w not in word_cnt:
                word_cnt[w] = 1
            else:
                word_cnt[w] += 1
    
    word_cnt_df = pd.DataFrame({'word': [k for k in word_cnt.keys()], 'cnt': [v for v in word_cnt.values()]})
    word_cnt_df.sort_values(by=['cnt'])
    vocab = {}
    target_words = word_cnt_df[word_cnt_df['cnt'] > 3]['word'].to_numpy()
    for word in target_words:
        if word not in vocab:
            vocab[word] = len(vocab)

    re_vocab = {}
    for word, i in vocab.items():
        re_vocab[i] = word        
    
    combinations = [list(itertools.combinations(words, 2)) for words in col_one_list]
    combination_matrix = np.zeros((len(vocab), len(vocab)))

    for tweet_comb in combinations:
        for comb in tweet_comb:
            if comb[0] in target_words and comb[1] in target_words:
                combination_matrix[vocab[comb[0]], vocab[comb[1]]] += 1
                combination_matrix[vocab[comb[1]], vocab[comb[0]]] += 1
        
    for i in range(len(vocab)):
            combination_matrix[i, i] /= 2
        
    jaccard_matrix = 1 - distance.cdist(combination_matrix, combination_matrix, 'jaccard')
    nodes = []
    for i in range(len(vocab)):
        for j in range(i+1, len(vocab)):
            jaccard = jaccard_matrix[i, j]
            if jaccard > 0:
                nodes.append([re_vocab[i], re_vocab[j], word_cnt[re_vocab[i]], word_cnt[re_vocab[j]], jaccard])
    
    G = nx.Graph()
    G.nodes(data=True)
    for pair in nodes:
        node_x, node_y, node_x_cnt, node_y_cnt, jaccard = pair[0], pair[1], pair[2], pair[3], pair[4]
        if not G.has_node(node_x):
            G.add_node(node_x, count=node_x_cnt)
        if not G.has_node(node_y):
            G.add_node(node_y, count=node_y_cnt)
        if not G.has_edge(node_x, node_y):
            G.add_edge(node_x, node_y, weight=jaccard)
        
    plt.figure(figsize=(20,15))
    pos = nx.spring_layout(G, k=0.1)

    node_size = [d['count']*150 for (n,d) in G.nodes(data=True)]
    nx.draw_networkx_nodes(G, pos, node_color='lightgreen', alpha=1.0, node_size=node_size)
    nx.draw_networkx_labels(G, pos)

    edge_width = [d['weight']*7 for (u,v,d) in G.edges(data=True)]
    nx.draw_networkx_edges(G, pos, alpha=0.3, edge_color='grey', width=edge_width)

    plt.axis('off')
    plt.savefig('images/energy'+str(ano)+'.png', dpi=300)
